[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/style-aligned-colab/blob/main/style_aligned_gradio_colab.ipynb)

In [ ]:
!pip install -q diffusers transformers accelerate peft gradio==3.50.2 mediapy einops

%cd /content
!git clone -b dev https://github.com/camenduru/style-aligned
%cd /content/style-aligned

from diffusers import StableDiffusionXLPipeline, DDIMScheduler
import torch
import mediapy
import sa_handler

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipeline = StableDiffusionXLPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, 
                                                     variant="fp16", use_safetensors=False, scheduler=scheduler).to("cuda")

handler = sa_handler.Handler(pipeline)
sa_args = sa_handler.StyleAlignedArgs(share_group_norm=False, share_layer_norm=False, share_attention=True, 
                                      adain_queries=True, adain_keys=True, adain_values=False)

handler.register(sa_args)

import os
import shlex
import subprocess
from pathlib import Path
from typing import Union

id_rsa_file = "/content/id_rsa"
id_rsa_pub_file = "/content/id_rsa.pub"
if os.path.exists(id_rsa_file):
    os.remove(id_rsa_file)
if os.path.exists(id_rsa_pub_file):
    os.remove(id_rsa_pub_file)

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

gen_key(id_rsa_file)

import threading
def tunnel():
  !ssh -R 80:127.0.0.1:7860 -o StrictHostKeyChecking=no -i /content/id_rsa remote.moe
threading.Thread(target=tunnel, daemon=True).start()

import gradio as gr

def generate(prompt):
    image = pipeline(prompt,).images[0]
    return image

with gr.Blocks(title=f"Realtime SDXL Turbo", css=".gradio-container {max-width: 544px !important}") as demo:
    with gr.Row():
      with gr.Column():
          textbox = gr.Textbox(show_label=False, value="a close-up picture of a fluffy cat")
          button = gr.Button()
    with gr.Row(variant="default"):
        output_image = gr.Image(
            show_label=False,
            type="pil",
            interactive=False,
            height=512,
            width=512,
            elem_id="output_image",
        )

    # textbox.change(fn=generate, inputs=[textbox], outputs=[output_image], show_progress=False)
    button.click(fn=generate, inputs=[textbox], outputs=[output_image], show_progress=False)

demo.queue().launch(inline=False, share=True, debug=True)